In [29]:
import requests
from bs4 import BeautifulSoup
import codecs

In [46]:
link = 'http://www.santostang.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'}
r = requests.get(link, headers = headers)
soup  = BeautifulSoup(r.text,"lxml")
title = soup.find("h1", class_="post-title").a.text.strip() # 使用 bs4 解析代码
print(title)# print the first title of the page
with codecs.open('save-files/title.txt','a+','utf-8') as f:
    f.write(title)

第四章 – 4.3 通过selenium 模拟浏览器抓取


# 抓取静态网页
### 定制 Requests

In [35]:
import requests
key_dict = {'key1': 'value1', 'key2': 'value2'}
r = requests.get('http://httpbin.org/get', params=key_dict,timeout = 20) #timeout,如果超过20s无响应就停止
print ("URL已经正确编码:", r.url)
print ("字符串方式的响应体: \n", r.text)

URL已经正确编码: http://httpbin.org/get?key1=value1&key2=value2
字符串方式的响应体: 
 {
  "args": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.20.0"
  }, 
  "origin": "172.113.239.90, 172.113.239.90", 
  "url": "https://httpbin.org/get?key1=value1&key2=value2"
}



In [32]:
import requests
key_dict = {'key1': 'value1', 'key2': 'value2'}
r = requests.post('http://httpbin.org/post', data=key_dict) # POST
print (r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "23", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.20.0"
  }, 
  "json": null, 
  "origin": "172.113.239.90, 172.113.239.90", 
  "url": "https://httpbin.org/post"
}



In [43]:
# 爬取豆瓣电影 Top250
import requests
from bs4 import BeautifulSoup
def get_movies():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'
              ,'Host':'movie.douban.com'}
    movie_list=[]
    for i in range(0,10):
        link = 'https://movie.douban.com/top250?start='+str(i*25)
        r = requests.get(link, headers = headers,timeout = 20)
        soup = BeautifulSoup(r.text,'lxml')
        div_list = soup.find_all('div',class_='hd')
        for each in div_list:
            movie_list.append(each.a.span.text.strip())
    return movie_list
movies = get_movies()
with codecs.open('save-files/douban250.txt','a+','utf-8')as f:
    f.write(str(movies))

# 抓取动态网页


In [24]:
import requests
import json

link = 'https://api-zero.livere.com/v1/comments/list?callback=jQuery11240178955723632786_1561587436044&limit=10&repSeq=4272904&requestPath=/v1/comments/list&consumerSeq=1020&livereSeq=28583&smartloginSeq=5154&_=1561587436046'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'}
r = requests.get(link,headers=headers)
print(r.text)


/**/ typeof jQuery11240178955723632786_1561587436044 === 'function' && jQuery11240178955723632786_1561587436044({"results":{"parents":[{"replySeq":37684295,"name":"风宇者","memberId":"oBVoaxKuZfl_rAbgoRqRXdGKK4Pw","memberIcon":"http://thirdwx.qlogo.cn/mmopen/vi_32/DYAIOgq83epXHLm9icdEzOibBmbzRT9NJbD4OtfIGVjY0gprLoftUu3SJxk6ImZyXBhffYRQjuxXibQyFgHqTAiaqw/132","memberUrl":"http://www.wechat.com","memberDomain":"wechat","good":0,"bad":0,"police":0,"parentSeq":37684295,"directSeq":0,"shortUrl":null,"title":"Hello world!","site":"http://www.santostang.com/2018/07/04/hello-world/","email":null,"ipAddress":"125.82.23.76","isMobile":"0","agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0","septSns":null,"targetService":null,"targetUserName":null,"info1":null,"info2":null,"info3":null,"image1":null,"image2":null,"image3":null,"link1":null,"link2":null,"link3":null,"isSecret":0,"isModified":0,"confirm":0,"subCount":1,"regdate":"2019-05-12T11:38:09.000Z","deletedDa

In [25]:
# 获取 json string
json_str =r.text
json_str=json_str[json_str.find('{'):-2] #从{ 左边第二个开始提取

json_data = json.loads(json_str)  
comment_list =json_data['results']['parents']

for each in comment_list:
    message = each['content']
    print(message)

简直有毒，这段代码有时候可以执行，有时候又失效了
from selenium import webdriver
driver = webdriver.Firefox(executable_path = r'C:/Users/Administrator/Desktop/geckodriver.exe')
driver.get('http://www.santostang.com/2018/07/04/hello-world/')
driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere']"))
comment = driver.find_element_by_css_selector('div.reply-content')
content = comment.find_element_by_tag_name('p')
print (content.text)
捣鼓了半天，终于把4.3弄出来了。有一些经验分享一下：（主要是第一个的问题，搞了很久，安装卸载程序好几次）
一个是Firefox的版本和Selenium的版本的问题。我最后安装的是Firefox的v55，然后Selenium装的是v3.5.0，因此geckodriver的版本也不是最新版（这个可以去它的官网上看，会说明适用什么版本的），这样就不会出现老师所说的还要补充自己电脑中geckodriver.exe程序的地址的步骤（我觉得这样会方便一点）。其实各个版本的对应关系还有很多，大家可以百度哈（我都是各种百度后解决的）
二是4.3中for循环中，有一行忘记缩进了（这个很明显哈，大家看到就知道了）
谢谢老师，开始学习
你好，请问为什么我使用书上第2章的爬虫代码时报错？能指导一下吗？我实在不明白soup.find().a.text.strip()中的a是什么意思。
你好，在做第三章爬取豆瓣top250电影时，提示：div_list = soup.find_all('div', class = 'hd')   这行代码语法错误。是因为class的原因吗？ 麻烦有时间指导一下，谢谢。
我遇到问题了，出现Webdrrive error,不知道怎么解决
正在学到第四章动态网页爬取。json解析部分。有没有一起学习讨论下

In [53]:
def single_page_comment():
    message =[]
    #the only difference between links is offset
    for page in range(1,20):
        link1 =  'https://api-zero.livere.com/v1/comments/list?callback=jQuery11240178955723632786_1561587436044&limit=10&offset='
        link2 = '&repSeq=4272904&requestPath=/v1/comments/list&consumerSeq=1020&livereSeq=28583&smartloginSeq=5154&_=1561587436046'
        page_str = str(page)
        link = link1 + page_str + link2

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'}
        r = requests.get(link,headers=headers)
        json_str =r.text
        json_str=json_str[json_str.find('{'):-2] #从{ 左边第二个开始提取

        json_data = json.loads(json_str)  
        comment_list =json_data['results']['parents']

        for each in comment_list:
            message.append(each['content'])
    return message
            
single_page_comment()

['简直有毒，这段代码有时候可以执行，有时候又失效了\nfrom selenium import webdriver\ndriver = webdriver.Firefox(executable_path = r\'C:/Users/Administrator/Desktop/geckodriver.exe\')\ndriver.get(\'http://www.santostang.com/2018/07/04/hello-world/\')\ndriver.switch_to.frame(driver.find_element_by_css_selector("iframe[title=\'livere\']"))\ncomment = driver.find_element_by_css_selector(\'div.reply-content\')\ncontent = comment.find_element_by_tag_name(\'p\')\nprint (content.text)',
 '捣鼓了半天，终于把4.3弄出来了。有一些经验分享一下：（主要是第一个的问题，搞了很久，安装卸载程序好几次）\n一个是Firefox的版本和Selenium的版本的问题。我最后安装的是Firefox的v55，然后Selenium装的是v3.5.0，因此geckodriver的版本也不是最新版（这个可以去它的官网上看，会说明适用什么版本的），这样就不会出现老师所说的还要补充自己电脑中geckodriver.exe程序的地址的步骤（我觉得这样会方便一点）。其实各个版本的对应关系还有很多，大家可以百度哈（我都是各种百度后解决的）\n二是4.3中for循环中，有一行忘记缩进了（这个很明显哈，大家看到就知道了）',
 '谢谢老师，开始学习',
 '你好，请问为什么我使用书上第2章的爬虫代码时报错？能指导一下吗？我实在不明白soup.find().a.text.strip()中的a是什么意思。',
 "你好，在做第三章爬取豆瓣top250电影时，提示：div_list = soup.find_all('div', class = 'hd')   这行代码语法错误。是因为class的原因吗？ 麻烦有时间指导一下，谢谢。",
 '我遇到问题了，出现Webdrrive error,

## 通过 selenium 模拟浏览器抓取

In [3]:
from selenium import webdriver
driver = webdriver.Firefox()
driver.get('http://www.santostang.com/2018/07/04/hello-world/')


In [6]:
# get the first comment 
comment = driver.find_element_by_css_selector('div.reply-content')
content = comment.find_element_by_tag_name('p')
print (content.text) 

简直有毒，这段代码有时候可以执行，有时候又失效了
from selenium import webdriver
driver = webdriver.Firefox(executable_path = r'C:/Users/Administrator/Desktop/geckodriver.exe')
driver.get('http://www.santostang.com/2018/07/04/hello-world/')
driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere']"))
comment = driver.find_element_by_css_selector('div.reply-content')
content = comment.find_element_by_tag_name('p')
print (content.text)


In [14]:
import time 
from selenium import webdriver
driver = webdriver.Firefox()
driver.get('http://www.santostang.com/2018/07/04/hello-world/')
driver.implicitly_wait(60)
time.sleep(5)

for i in range(0,5):
    # 下滑到页面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #转换iframe，找到【查找更多】，点击
    driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere']"))
    load_more = driver.find_element_by_css_selector('button.more-btn')
    # 把iframe 转回去
    driver.switch_to.default_content()
    time.sleep(2)
    
driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere']"))
comments= driver.find_elements_by_css_selector('div.reply-content')
for each in comments:
    content = each.find_element_by_tag_name('p')
    print(content.text)

简直有毒，这段代码有时候可以执行，有时候又失效了
from selenium import webdriver
driver = webdriver.Firefox(executable_path = r'C:/Users/Administrator/Desktop/geckodriver.exe')
driver.get('http://www.santostang.com/2018/07/04/hello-world/')
driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere']"))
comment = driver.find_element_by_css_selector('div.reply-content')
content = comment.find_element_by_tag_name('p')
print (content.text)
@风宇者 感觉是因为网速不够快没加载出来程序就提取了，应该让他睡几秒。
捣鼓了半天，终于把4.3弄出来了。有一些经验分享一下：（主要是第一个的问题，搞了很久，安装卸载程序好几次）
一个是Firefox的版本和Selenium的版本的问题。我最后安装的是Firefox的v55，然后Selenium装的是v3.5.0，因此geckodriver的版本也不是最新版（这个可以去它的官网上看，会说明适用什么版本的），这样就不会出现老师所说的还要补充自己电脑中geckodriver.exe程序的地址的步骤（我觉得这样会方便一点）。其实各个版本的对应关系还有很多，大家可以百度哈（我都是各种百度后解决的）
二是4.3中for循环中，有一行忘记缩进了（这个很明显哈，大家看到就知道了）
谢谢老师，开始学习
你好，请问为什么我使用书上第2章的爬虫代码时报错？能指导一下吗？我实在不明白soup.find().a.text.strip()中的a是什么意思。
@　Burning h1标签不是hi标签。你查hi下面他就提示你找不到a标签
你好，在做第三章爬取豆瓣top250电影时，提示：div_list = soup.find_all('div', class = 'hd') 这行代码语法错误。是因为class的原因吗？ 麻烦有时间指导

In [ ]:
"""
控制selenium 抓取的速度：
- 控制 css 的加载
- 控制图片显示
- 控制 js 的运行
"""

In [ ]:
# 控制 css 加载
fp = webdriver.FirefoxProfile()
fp.set_preference('permissions.default.stylesheet',2)
# 控制图片显示
fp.set_preference('permissions.default.image',2)
# 控制 js 的运行
fp.set_preference('javascript.enabled',False)

driver = webdriver.Firefox(firefox_profile=fp)

In [38]:
# 爬取 Airbnb 青岛17页房源
import codecs
import time
from selenium import webdriver

#fp = webdriver.FirefoxProfile()
#fp.set_preference('permissions.default.stylesheet',2)
fp.set_preference('permissions.default.image',2)
#fp.set_preference('javascript.enabled',False)
driver = webdriver.Firefox()

driver.get("https://zh.airbnb.com/s/Qingdao--China/homes")
print('testetste')
#找到页面中所有的出租房
rent_list = driver.find_elements_by_css_selector('div._gig1e7')
for each in rent_list:
    
    price = each.find_element_by_css_selector('div._18gk84h')
    #price = price.replace('每晚','')
    comment = each.find_element_by_css_selector('span._1cy09umr')
    details = each.find_element_by_css_selector('div._qhtkbey').text
    Htype = each.find_element_by_css_selector('span._fk7kh10')
    house_type = Htype.split('·')[0]
    bed_number = Htype.split('·')[1]
    
    with open('save-files/airbnb-test.txt','a+')as f:
       
       f.write(comment,details,price,house_type,bed_number)
time.sleep(5)

testetste


"    \n    with open('save-files/airbnb-test.txt','a+')as f:\n        f.write(price)\n"

## BeautifulSoup 

In [42]:
import requests
from bs4 import BeautifulSoup
link = 'http://www.santostang.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'}
r =requests.get(link, headers=headers)
soup = BeautifulSoup(r.text,'html.parser')
first_title = soup.find('h1',class_='post-title').a.text.strip() # first title 
title_list = soup.find_all('h1',class_='post-title') #all titles
for i in range(len(title_list)):
    title = title_list[i].a.text.strip()
    print(title)
    

第四章 – 4.3 通过selenium 模拟浏览器抓取
第四章 – 4.2 解析真实地址抓取
第四章- 动态网页抓取 (解析真实地址 + selenium)
Hello world!
